Modulo 0: Configuracion Inicial y Dependencias

In [197]:
# ===== CONFIGURACIÓN INICIAL =====
import re
import unicodedata
import json
import numpy as np
import torch
from datetime import datetime
from typing import Dict, List, Optional, Tuple
import warnings
import subprocess
import sys

# Configurar advertencias
warnings.filterwarnings('ignore')

# Configurar dispositivo para PyTorch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ===== INSTALACIÓN DE DEPENDENCIAS =====
# Instalar spaCy y modelo de español
try:
    import spacy
    nlp = spacy.load("es_core_news_md")
except:
    subprocess.check_call([sys.executable, "-m", "spacy", "download", "es_core_news_md"])
    nlp = spacy.load("es_core_news_md")

# Instalar NLTK y descargar recursos
try:
    import nltk
    from nltk.tokenize import word_tokenize, sent_tokenize
    from nltk.stem import SnowballStemmer, WordNetLemmatizer
    from nltk.corpus import stopwords
    nltk.download('punkt')
    nltk.download('wordnet')
    nltk.download('stopwords')
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "nltk"])
    import nltk
    nltk.download('punkt')
    nltk.download('wordnet')
    nltk.download('stopwords')

# Instalar transformers para BERT
try:
    from transformers import BertTokenizer, BertModel
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "transformers"])
    from transformers import BertTokenizer, BertModel

# Texto de ejemplo para todo el procesamiento
TEXTO_EJEMPLO = """
El cambio climático representa uno de los mayores desafíos ambientales del siglo XXI.
La deforestación, la contaminación del aire y el uso excesivo de combustibles fósiles
están afectando gravemente nuestro planeta. Es fundamental implementar energías renovables
y promover prácticas sostenibles para proteger el medio ambiente para las futuras generaciones.
Las ONGs trabajan para proteger la biodiversidad, pero necesitamos más acciones concretas.
Visita https://ejemplo.com para más información o contacta a info@medioambiente.org.
"""

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adrib\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\adrib\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\adrib\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Modulo 1: Ingesta de Texto

In [198]:
# ===== MÓDULO 1: INGESTA DEL TEXTO =====
class TextIngestion:
    """
    Módulo para la ingesta de texto desde diferentes fuentes.
    Permite entrada manual, desde archivos o URLs.
    """
    
    def __init__(self):
        self.supported_formats = ['txt', 'csv', 'json']
    
    def ingest_manual_text(self, text):
        """
        Ingesta de texto manual directo
        """
        if not isinstance(text, str):
            raise ValueError("El texto debe ser una cadena de caracteres")
        
        return {
            'original_text': text,
            'source': 'manual',
            'length': len(text),
            'word_count': len(text.split())
        }
    
    def ingest_from_file(self, file_path):
        """
        Ingesta de texto desde archivo
        """
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()
            
            return {
                'original_text': text,
                'source': f'file: {file_path}',
                'length': len(text),
                'word_count': len(text.split())
            }
        except Exception as e:
            raise Exception(f"Error al leer archivo: {str(e)}")
    
    def validate_text(self, text_data):
        """
        Validación básica del texto ingresado
        """
        text = text_data['original_text']
        
        validations = {
            'is_empty': len(text.strip()) == 0,
            'min_length': len(text) >= 10,
            'has_letters': bool(re.search(r'[a-zA-ZáéíóúÁÉÍÓÚñÑ]', text)),
            'encoding_issues': '�' in text
        }
        
        return validations

Modulo 2: Limpieza Basica

In [199]:
# ===== MÓDULO 2: LIMPIEZA BÁSICA =====
class TextCleaner:
    def __init__(self):
        self.url_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
        self.email_pattern = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b')
        self.phone_pattern = re.compile(r'(\+?[0-9]{1,3}[-.\s]?)?(\(?[0-9]{3}\)?[-.\s]?[0-9]{3}[-.\s]?[0-9]{4})')
        self.html_pattern = re.compile(r'<[^>]+>')
        self.special_chars_pattern = re.compile(r'[^\w\s\.\,\;\:\!\?\-\(\)áéíóúÁÉÍÓÚñÑüÜ]')
        self.multiple_spaces_pattern = re.compile(r'\s+')
        self.multiple_punctuation_pattern = re.compile(r'([.!?]){2,}')
    
    def remove_urls(self, text: str) -> str:
        return self.url_pattern.sub('', text)
    
    def remove_emails(self, text: str) -> str:
        return self.email_pattern.sub('', text)
    
    def remove_phone_numbers(self, text: str) -> str:
        return self.phone_pattern.sub('', text)
    
    def remove_html_tags(self, text: str) -> str:
        return self.html_pattern.sub('', text)
    
    def remove_special_characters(self, text: str, keep_punctuation: bool = True) -> str:
        if keep_punctuation:
            cleaned = re.sub(r'[^\w\s\.\,\;\:\!\?\-\(\)áéíóúÁÉÍÓÚñÑüÜ]', ' ', text)
        else:
            cleaned = re.sub(r'[^\w\s áéíóúÁÉÍÓÚñÑüÜ]', ' ', text)
        return cleaned
    
    def normalize_whitespace(self, text: str) -> str:
        text = self.multiple_spaces_pattern.sub(' ', text)
        return text.strip()
    
    def normalize_punctuation(self, text: str) -> str:
        return self.multiple_punctuation_pattern.sub(r'\1', text)
    
    def remove_extra_newlines(self, text: str) -> str:
        text = re.sub(r'\n+', '\n', text)
        text = re.sub(r'\n', ' ', text)
        return text
    
    def clean_encoding_issues(self, text: str) -> str:
        encoding_fixes = {
            'Ã¡': 'á', 'Ã©': 'é', 'Ã­': 'í', 'Ã³': 'ó', 'Ãº': 'ú',
            'Ã±': 'ñ', 'Ã¼': 'ü', 'Â': '', 'â€™': "'", 'â€œ': '"',
            'â€': '"', 'â€¦': '...', 'â€"': '-', '�': ''
        }
        
        for wrong, correct in encoding_fixes.items():
            text = text.replace(wrong, correct)
        
        return text
    
    def basic_clean(self, text: str, options: Dict = None) -> Dict:
        if options is None:
            options = {
                'remove_urls': True,
                'remove_emails': True,
                'remove_phones': True,
                'remove_html': True,
                'remove_special_chars': True,
                'keep_punctuation': True,
                'normalize_whitespace': True,
                'normalize_punctuation': True,
                'remove_newlines': True,
                'fix_encoding': True
            }
        
        original_text = text
        original_length = len(text)
        
        if options.get('fix_encoding', True):
            text = self.clean_encoding_issues(text)
        
        if options.get('remove_html', True):
            text = self.remove_html_tags(text)
        
        if options.get('remove_urls', True):
            text = self.remove_urls(text)
        
        if options.get('remove_emails', True):
            text = self.remove_emails(text)
        
        if options.get('remove_phones', True):
            text = self.remove_phone_numbers(text)
        
        if options.get('remove_newlines', True):
            text = self.remove_extra_newlines(text)
        
        if options.get('remove_special_chars', True):
            text = self.remove_special_characters(text, options.get('keep_punctuation', True))
        
        if options.get('normalize_punctuation', True):
            text = self.normalize_punctuation(text)
        
        if options.get('normalize_whitespace', True):
            text = self.normalize_whitespace(text)
        
        cleaned_length = len(text)
        reduction_percentage = ((original_length - cleaned_length) / original_length) * 100 if original_length > 0 else 0
        
        return {
            'original_text': original_text,
            'cleaned_text': text,
            'original_length': original_length,
            'cleaned_length': cleaned_length,
            'reduction_percentage': round(reduction_percentage, 2),
            'cleaning_options': options
        }

Modulo 3: Tokenizacion

In [200]:
# ===== MÓDULO 3: TOKENIZACIÓN =====
class TextTokenizer:
    def __init__(self):
        self.nlp = nlp
        self.sentence_endings = re.compile(r'[.!?]+')
        self.word_pattern = re.compile(r'\b\w+\b')
        self.punctuation_pattern = re.compile(r'[^\w\s]')
    
    def tokenize_sentences(self, text: str, method: str = 'nltk') -> List[str]:
        if method == 'nltk':
            sentences = sent_tokenize(text, language='spanish')
        elif method == 'spacy':
            doc = self.nlp(text)
            sentences = [sent.text.strip() for sent in doc.sents]
        elif method == 'regex':
            sentences = self.sentence_endings.split(text)
            sentences = [s.strip() for s in sentences if s.strip()]
        else:
            raise ValueError("Método debe ser 'nltk', 'spacy' o 'regex'")
        
        return [s for s in sentences if len(s.strip()) > 0]
    
    def tokenize_words(self, text: str, method: str = 'nltk') -> List[str]:
        if method == 'nltk':
            words = word_tokenize(text, language='spanish')
        elif method == 'spacy':
            doc = self.nlp(text)
            words = [token.text for token in doc if not token.is_space]
        elif method == 'regex':
            words = self.word_pattern.findall(text)
        else:
            raise ValueError("Método debe ser 'nltk', 'spacy' o 'regex'")
        
        return words
    
    def tokenize_paragraphs(self, text: str) -> List[str]:
        paragraphs = re.split(r'\n\s*\n', text)
        return [p.strip() for p in paragraphs if p.strip()]
    
    def advanced_tokenization(self, text: str) -> Dict:
        doc = self.nlp(text)
        
        tokens_info = []
        for token in doc:
            if not token.is_space:
                token_info = {
                    'text': token.text,
                    'lemma': token.lemma_,
                    'pos': token.pos_,
                    'tag': token.tag_,
                    'is_alpha': token.is_alpha,
                    'is_stop': token.is_stop,
                    'is_punct': token.is_punct,
                    'is_digit': token.is_digit,
                    'shape': token.shape_,
                    'is_title': token.is_title,
                    'is_lower': token.is_lower,
                    'is_upper': token.is_upper
                }
                tokens_info.append(token_info)
        
        return {
            'tokens': tokens_info,
            'total_tokens': len(tokens_info),
            'sentences': [sent.text for sent in doc.sents],
            'entities': [(ent.text, ent.label_) for ent in doc.ents]
        }
    
    def tokenize_by_pos(self, text: str) -> Dict[str, List[str]]:
        doc = self.nlp(text)
        
        pos_groups = {
            'sustantivos': [],
            'verbos': [],
            'adjetivos': [],
            'adverbios': [],
            'preposiciones': [],
            'conjunciones': [],
            'determinantes': [],
            'pronombres': [],
            'otros': []
        }
        
        pos_mapping = {
            'NOUN': 'sustantivos',
            'VERB': 'verbos',
            'ADJ': 'adjetivos',
            'ADV': 'adverbios',
            'ADP': 'preposiciones',
            'CONJ': 'conjunciones',
            'CCONJ': 'conjunciones',
            'DET': 'determinantes',
            'PRON': 'pronombres'
        }
        
        for token in doc:
            if token.is_alpha and not token.is_stop:
                category = pos_mapping.get(token.pos_, 'otros')
                pos_groups[category].append(token.lemma_.lower())
        
        for category in pos_groups:
            pos_groups[category] = list(dict.fromkeys(pos_groups[category]))
        
        return pos_groups
    
    def extract_environmental_terms(self, text: str) -> Dict[str, List[str]]:
        doc = self.nlp(text)
        
        environmental_categories = {
            'problemas_ambientales': [],
            'soluciones': [],
            'recursos_naturales': [],
            'energia': [],
            'contaminacion': [],
            'conservacion': []
        }
        
        environmental_terms = {
            'problemas_ambientales': [
                'cambio climático', 'calentamiento global', 'deforestación', 
                'contaminación', 'extinción', 'desertificación', 'erosión'
            ],
            'soluciones': [
                'reciclaje', 'energía renovable', 'sostenible', 'sustentable',
                'conservación', 'reforestación', 'eficiencia energética'
            ],
            'recursos_naturales': [
                'agua', 'bosque', 'océano', 'biodiversidad', 'ecosistema',
                'fauna', 'flora', 'selva', 'río', 'lago'
            ],
            'energia': [
                'solar', 'eólica', 'hidroeléctrica', 'geotérmica', 
                'biomasa', 'combustible fósil', 'petróleo', 'carbón'
            ],
            'contaminacion': [
                'emisiones', 'gases', 'residuos', 'basura', 'tóxicos',
                'plástico', 'químicos', 'desechos'
            ],
            'conservacion': [
                'protección', 'preservación', 'área protegida', 'parque nacional',
                'reserva', 'santuario', 'hábitat'
            ]
        }
        
        text_lower = text.lower()
        
        for category, terms in environmental_terms.items():
            for term in terms:
                if term in text_lower:
                    environmental_categories[category].append(term)
        
        return environmental_categories
    
    def tokenization_statistics(self, text: str) -> Dict:
        sentences = self.tokenize_sentences(text, 'spacy')
        words = self.tokenize_words(text, 'spacy')
        paragraphs = self.tokenize_paragraphs(text)
        advanced_tokens = self.advanced_tokenization(text)
        pos_groups = self.tokenize_by_pos(text)
        env_terms = self.extract_environmental_terms(text)
        
        word_lengths = [len(word) for word in words if word.isalpha()]
        sentence_lengths = [len(sent.split()) for sent in sentences]
        
        stats = {
            'total_characters': len(text),
            'total_words': len(words),
            'total_sentences': len(sentences),
            'total_paragraphs': len(paragraphs),
            'unique_words': len(set(word.lower() for word in words if word.isalpha())),
            'avg_word_length': np.mean(word_lengths) if word_lengths else 0,
            'avg_sentence_length': np.mean(sentence_lengths) if sentence_lengths else 0,
            'pos_distribution': {k: len(v) for k, v in pos_groups.items()},
            'environmental_terms_count': sum(len(terms) for terms in env_terms.values()),
            'entities_found': len(advanced_tokens['entities']),
            'lexical_diversity': len(set(word.lower() for word in words if word.isalpha())) / len(words) if words else 0
        }
        
        return stats

Modulo 4: Normalizacion

In [201]:
# ===== MÓDULO 4: NORMALIZACIÓN =====
class TextNormalizer:
    def __init__(self):
        self.contractions = {
            'del': 'de el', 'al': 'a el', 'pa': 'para', 'pal': 'para el',
            'pá': 'para', 'q': 'que', 'xq': 'porque', 'porq': 'porque',
            'x': 'por', 'tb': 'también', 'tmb': 'también', 'tbn': 'también',
            'pq': 'porque', 'xk': 'porque', 'k': 'que', 'dnd': 'donde',
            'dónde': 'donde', 'cuándo': 'cuando', 'cómo': 'como',
            'qué': 'que', 'cuál': 'cual', 'cuáles': 'cuales',
            'quién': 'quien', 'quiénes': 'quienes'
        }
        
        self.abbreviations = {
            'dr.': 'doctor', 'dra.': 'doctora', 'sr.': 'señor',
            'sra.': 'señora', 'srta.': 'señorita', 'prof.': 'profesor',
            'profa.': 'profesora', 'ing.': 'ingeniero', 'lic.': 'licenciado',
            'etc.': 'etcétera', 'vs.': 'versus', 'ej.': 'ejemplo',
            'p.ej.': 'por ejemplo', 'i.e.': 'es decir', 'e.g.': 'por ejemplo',
            'aprox.': 'aproximadamente', 'máx.': 'máximo', 'mín.': 'mínimo',
            'kg.': 'kilogramos', 'km.': 'kilómetros', 'm.': 'metros',
            'cm.': 'centímetros', 'mm.': 'milímetros', 'co2': 'dióxido de carbono',
            'ong': 'organización no gubernamental', 'onu': 'organización de las naciones unidas'
        }
        
        self.number_words = {
            'cero': '0', 'uno': '1', 'dos': '2', 'tres': '3', 'cuatro': '4',
            'cinco': '5', 'seis': '6', 'siete': '7', 'ocho': '8', 'nueve': '9',
            'diez': '10', 'once': '11', 'doce': '12', 'trece': '13', 'catorce': '14',
            'quince': '15', 'dieciséis': '16', 'diecisiete': '17', 'dieciocho': '18',
            'diecinueve': '19', 'veinte': '20', 'treinta': '30', 'cuarenta': '40',
            'cincuenta': '50', 'sesenta': '60', 'setenta': '70', 'ochenta': '80',
            'noventa': '90', 'cien': '100', 'mil': '1000', 'millón': '1000000'
        }
        
        self.env_normalizations = {
            'co2': 'dióxido de carbono', 'co₂': 'dióxido de carbono',
            'ch4': 'metano', 'ch₄': 'metano', 'n2o': 'óxido nitroso',
            'n₂o': 'óxido nitroso', 'ghg': 'gases de efecto invernadero',
            'gei': 'gases de efecto invernadero', 'renewable energy': 'energía renovable',
            'green house': 'efecto invernadero', 'global warming': 'calentamiento global',
            'climate change': 'cambio climático', 'sustainable development': 'desarrollo sostenible',
            'carbon footprint': 'huella de carbono', 'biodiversity': 'biodiversidad',
            'ecosystem': 'ecosistema', 'deforestation': 'deforestación',
            'reforestation': 'reforestación'
        }
    
    def to_lowercase(self, tokens: List[str]) -> List[str]:
        return [token.lower() for token in tokens]
    
    def remove_accents(self, tokens: List[str]) -> List[str]:
        normalized_tokens = []
        for token in tokens:
            text_nfd = unicodedata.normalize('NFD', token)
            text_without_accents = ''.join(
                char for char in text_nfd 
                if unicodedata.category(char) != 'Mn'
            )
            normalized_tokens.append(text_without_accents)
        return normalized_tokens
    
    def expand_contractions(self, tokens: List[str]) -> List[str]:
        expanded_tokens = []
        for token in tokens:
            token_lower = token.lower()
            if token_lower in self.contractions:
                expansion = self.contractions[token_lower]
                if ' ' in expansion:
                    expanded_tokens.extend(expansion.split())
                else:
                    expanded_tokens.append(expansion)
            else:
                expanded_tokens.append(token)
        return expanded_tokens
    
    def expand_abbreviations(self, tokens: List[str]) -> List[str]:
        expanded_tokens = []
        for token in tokens:
            token_lower = token.lower()
            if token_lower in self.abbreviations:
                expansion = self.abbreviations[token_lower]
                if ' ' in expansion:
                    expanded_tokens.extend(expansion.split())
                else:
                    expanded_tokens.append(expansion)
            else:
                expanded_tokens.append(token)
        return expanded_tokens
    
    def normalize_numbers(self, tokens: List[str], strategy: str = 'keep') -> List[str]:
        if strategy == 'remove':
            return [token for token in tokens if not token.isdigit()]
        elif strategy == 'words_to_digits':
            normalized_tokens = []
            for token in tokens:
                token_lower = token.lower()
                if token_lower in self.number_words:
                    normalized_tokens.append(self.number_words[token_lower])
                else:
                    normalized_tokens.append(token)
            return normalized_tokens
        elif strategy == 'digits_to_words':
            digit_to_word = {v: k for k, v in self.number_words.items() if int(v) <= 20}
            normalized_tokens = []
            for token in tokens:
                if token.isdigit() and token in digit_to_word:
                    normalized_tokens.append(digit_to_word[token])
                else:
                    normalized_tokens.append(token)
            return normalized_tokens
        else:
            return tokens
    
    def normalize_case_patterns(self, tokens: List[str]) -> List[str]:
        normalized_tokens = []
        for token in tokens:
            if token.isupper() and len(token) > 1:
                normalized_tokens.append(token.lower())
            elif token.islower():
                normalized_tokens.append(token)
            else:
                normalized_tokens.append(token)
        return normalized_tokens
    
    def normalize_environmental_terms(self, tokens: List[str]) -> List[str]:
        text = ' '.join(tokens).lower()
        
        for term, normalized in self.env_normalizations.items():
            text = re.sub(r'\b' + re.escape(term) + r'\b', normalized, text)
        
        return text.split()
    
    def normalize_tokens(self, tokens: List[str], options: Dict = None) -> Dict:
        if options is None:
            options = {
                'to_lowercase': True,
                'remove_accents': True,
                'expand_contractions': True,
                'expand_abbreviations': True,
                'normalize_numbers': 'keep',
                'normalize_case_patterns': True,
                'normalize_environmental_terms': True
            }
        
        original_tokens = tokens.copy()
        steps_applied = []
        
        if options.get('normalize_case_patterns', True):
            tokens = self.normalize_case_patterns(tokens)
            steps_applied.append('case_patterns')
        
        if options.get('to_lowercase', True):
            tokens = self.to_lowercase(tokens)
            steps_applied.append('lowercase')
        
        if options.get('expand_contractions', True):
            tokens = self.expand_contractions(tokens)
            steps_applied.append('contractions')
        
        if options.get('expand_abbreviations', True):
            tokens = self.expand_abbreviations(tokens)
            steps_applied.append('abbreviations')
        
        if options.get('normalize_environmental_terms', True):
            tokens = self.normalize_environmental_terms(tokens)
            steps_applied.append('environmental_terms')
        
        if options.get('normalize_numbers', 'keep') != 'keep':
            tokens = self.normalize_numbers(tokens, options['normalize_numbers'])
            steps_applied.append('numbers')
        
        if options.get('remove_accents', True):
            tokens = self.remove_accents(tokens)
            steps_applied.append('accents')
        
        return {
            'original_tokens': original_tokens,
            'normalized_tokens': tokens,
            'original_count': len(original_tokens),
            'normalized_count': len(tokens),
            'token_count_change': len(tokens) - len(original_tokens),
            'steps_applied': steps_applied,
            'normalization_options': options
        }
    
    def normalize_tokenized_text(self, tokenization_result: Dict, options: Dict = None) -> Dict:
        if 'tokens' in tokenization_result:
            tokens = [token['text'] for token in tokenization_result['tokens']]
        elif isinstance(tokenization_result, list):
            tokens = tokenization_result
        else:
            tokens = tokenization_result.get('words', [])
            if not tokens:
                raise ValueError("No se pudieron extraer tokens del resultado de tokenización")
        
        normalization_result = self.normalize_tokens(tokens, options)
        
        if 'tokens' in tokenization_result and isinstance(tokenization_result['tokens'], list):
            normalized_tokens_info = []
            
            for i, token_info in enumerate(tokenization_result['tokens']):
                if i < len(normalization_result['normalized_tokens']):
                    normalized_token = normalization_result['normalized_tokens'][i]
                    normalized_tokens_info.append({
                        **token_info,
                        'original_text': token_info['text'],
                        'text': normalized_token,
                        'normalized': True
                    })
            
            normalization_result['normalized_tokens_info'] = normalized_tokens_info
        
        return normalization_result

Modulo 5: Eliminacion del Ruido

In [202]:
# ===== MÓDULO 5: ELIMINACIÓN DE RUIDO (CORREGIDO) =====

class NoiseRemover:
    """
    Módulo para la eliminación de ruido del texto.
    Elimina stopwords, palabras muy frecuentes/raras, contenido irrelevante
    y ruido específico del dominio de manera inteligente y menos agresiva.
    """
    
    def __init__(self):
        # Stopwords básicas en español (más conservadoras)
        try:
            self.spanish_stopwords = set(stopwords.words('spanish'))
        except:
            # Fallback si NLTK no está disponible
            self.spanish_stopwords = {
                'el', 'la', 'de', 'que', 'y', 'a', 'en', 'un', 'es', 'se',
                'no', 'te', 'lo', 'le', 'da', 'su', 'por', 'son', 'con', 'para',
                'al', 'del', 'los', 'las', 'una', 'como', 'pero', 'sus', 'me',
                'hasta', 'hay', 'donde', 'han', 'quien', 'están', 'estado',
                'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni',
                'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto',
                'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras',
                'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'nada',
                'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas',
                'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus',
                'ellas', 'nosotras', 'vosotros', 'vosotras', 'os', 'mío', 'mía',
                'míos', 'mías', 'tuyo', 'tuya', 'tuyos', 'tuyas', 'suyo', 'suya',
                'suyos', 'suyas', 'nuestro', 'nuestra', 'nuestros', 'nuestras',
                'vuestro', 'vuestra', 'vuestros', 'vuestras', 'esos', 'esas'
            }
        
        # Stopwords adicionales personalizadas (reducidas y más específicas)
        self.custom_stopwords = {
            # Conectores básicos que realmente no aportan significado
            'pues', 'bueno', 'entonces', 'así', 'ahora', 'luego', 'después',
            'antes', 'mientras', 'durante', 'mediante', 'según', 'incluso',
            'sino', 'aunque', 'sin embargo', 'no obstante', 'por tanto', 
            'por consiguiente', 'en consecuencia',
            
            # Palabras de relleno muy generales
            'cosa', 'cosas', 'algo', 'nada', 'todo', 'todos', 'todas',
            'bastante', 'demasiado', 'suficiente',
            
            # Palabras muy generales que no aportan contexto específico
            'forma', 'manera', 'modo', 'tipo', 'tipos', 'clase', 'clases',
            'parte', 'partes', 'lado', 'lados', 'vez', 'veces', 'momento',
            'momentos', 'tiempo', 'tiempos', 'lugar', 'lugares', 'caso', 'casos'
        }
        
        # Stopwords específicas para contexto ambiental (muy reducidas)
        self.environmental_stopwords = {
            'tema', 'temas', 'situación', 'situaciones',
            'aspecto', 'aspectos', 'punto', 'puntos', 
            'cuestión', 'cuestiones', 'asunto', 'asuntos'
        }
        
        # Combinar stopwords de manera inteligente
        self.basic_stopwords = self.spanish_stopwords
        self.extended_stopwords = (self.spanish_stopwords | 
                                 self.custom_stopwords | 
                                 self.environmental_stopwords)
        
        # Patrones de ruido común (más específicos)
        self.noise_patterns = [
            r'\b\w{1,2}\b',  # Palabras muy cortas (1-2 caracteres)
            r'[^\w\s\.\,\;\:\!\?\-\(\)áéíóúÁÉÍÓÚñÑüÜ]',  # Caracteres especiales
        ]
        
        # Palabras de ruido específicas (interjecciones y muletillas)
        self.noise_words = {
            'mm', 'hmm', 'eh', 'ah', 'oh', 'uh', 'um', 'er',
            'ok', 'okay', 'je', 'ja', 'jaja', 'jeje', 'jajaja',
            'etc', 'etcetera', 'bla', 'blah', 'ajá', 'aja'
        }
        
        # Palabras ambientales importantes que NUNCA deben eliminarse
        self.environmental_keywords = {
            'ambiente', 'ambiental', 'ambientales', 'ecológico', 'ecológica',
            'sostenible', 'sustentable', 'verde', 'limpio', 'limpia',
            'renovable', 'renovables', 'conservación', 'biodiversidad',
            'clima', 'climático', 'climática', 'carbono', 'emisiones',
            'contaminación', 'contaminante', 'reciclaje', 'reciclar',
            'energía', 'energías', 'natural', 'naturales', 'planeta',
            'tierra', 'agua', 'aire', 'bosque', 'bosques', 'océano',
            'océanos', 'plantas', 'animales', 'especies', 'ecosistema',
            'ecosistemas', 'deforestación', 'reforestación', 'solar',
            'eólica', 'hidroeléctrica', 'geotérmica', 'biomasa',
            'combustible', 'combustibles', 'fósil', 'fósiles', 'petróleo',
            'carbón', 'gas', 'gases', 'residuos', 'basura', 'tóxicos',
            'químicos', 'desechos', 'protección', 'preservación',
            'calentamiento', 'global', 'efecto', 'invernadero'
        }
    
    def _is_important_word(self, word: str) -> bool:
        """Determina si una palabra es importante y no debe eliminarse"""
        word_lower = word.lower()
        
        # Palabras ambientales importantes
        if word_lower in self.environmental_keywords:
            return True
        
        # Palabras con significado específico (sustantivos, verbos, adjetivos importantes)
        if len(word) >= 4 and word.isalpha():
            return True
        
        # Números pueden ser importantes
        if word.isdigit():
            return True
        
        return False
    
    def remove_stopwords(self, text: str, custom_stopwords: set = None, aggressive: bool = False) -> str:
        """
        Elimina stopwords del texto de manera inteligente
        
        Args:
            text: Texto a procesar
            custom_stopwords: Stopwords adicionales personalizadas
            aggressive: Si True, usa eliminación más agresiva
        
        Returns:
            Texto sin stopwords
        """
        if not text or len(text.strip()) == 0:
            return text
        
        words = text.split()
        original_word_count = len(words)
        
        # Seleccionar conjunto de stopwords según el contexto
        if aggressive and original_word_count > 15:
            stopwords_to_use = self.extended_stopwords.copy()
        else:
            stopwords_to_use = self.basic_stopwords.copy()
        
        if custom_stopwords:
            stopwords_to_use.update(custom_stopwords)
        
        # Filtrar palabras manteniendo las importantes
        filtered_words = []
        for word in words:
            word_lower = word.lower()
            
            # Mantener palabras importantes sin importar si son stopwords
            if self._is_important_word(word):
                filtered_words.append(word)
            # Eliminar solo si es stopword y no es importante
            elif word_lower not in stopwords_to_use:
                filtered_words.append(word)
        
        # Verificar que no hayamos eliminado demasiadas palabras
        if len(filtered_words) < original_word_count * 0.4:  # Si quedan menos del 40%
            print(f"⚠️ Eliminación de stopwords demasiado agresiva ({len(filtered_words)}/{original_word_count}), usando solo stopwords básicas")
            # Intentar con stopwords más básicas
            filtered_words = []
            for word in words:
                if word.lower() not in self.basic_stopwords or self._is_important_word(word):
                    filtered_words.append(word)
        
        # Si aún es muy agresivo, devolver texto original
        if len(filtered_words) < original_word_count * 0.3:
            print(f"⚠️ Eliminación aún muy agresiva, devolviendo texto original")
            return text
        
        result = ' '.join(filtered_words) if filtered_words else text
        return result if result.strip() else text
    
    def remove_by_frequency(self, text: str, min_freq: int = 1, max_freq_ratio: float = 0.7) -> Dict:
        """
        Elimina palabras muy raras o muy frecuentes (versión menos agresiva)
        
        Args:
            text: Texto a procesar
            min_freq: Frecuencia mínima para mantener una palabra (reducido a 1)
            max_freq_ratio: Ratio máximo de frecuencia (aumentado a 0.7 = 70% del total)
        
        Returns:
            Diccionario con texto filtrado y estadísticas
        """
        if not text or len(text.strip()) == 0:
            return {
                'filtered_text': text,
                'original_word_count': 0,
                'filtered_word_count': 0,
                'rare_words_removed': 0,
                'frequent_words_removed': 0,
                'rare_words': [],
                'frequent_words': []
            }
        
        words = text.split()
        word_freq = {}
        
        # Contar frecuencias
        for word in words:
            word_lower = word.lower()
            word_freq[word_lower] = word_freq.get(word_lower, 0) + 1
        
        total_words = len(words)
        max_freq = int(total_words * max_freq_ratio)
        
        # Identificar palabras a eliminar (más conservador)
        rare_words = {word for word, freq in word_freq.items() 
                     if freq < min_freq and not self._is_important_word(word)}
        frequent_words = {word for word, freq in word_freq.items() 
                         if freq > max_freq and not self._is_important_word(word)}
        words_to_remove = rare_words | frequent_words
        
        # Filtrar palabras
        filtered_words = [
            word for word in words 
            if word.lower() not in words_to_remove
        ]
        
        # Verificar que no hayamos eliminado demasiado
        if len(filtered_words) < len(words) * 0.5:
            print("⚠️ Filtrado por frecuencia demasiado agresivo, devolviendo texto original")
            filtered_words = words
            rare_words = set()
            frequent_words = set()
        
        return {
            'filtered_text': ' '.join(filtered_words),
            'original_word_count': len(words),
            'filtered_word_count': len(filtered_words),
            'rare_words_removed': len(rare_words),
            'frequent_words_removed': len(frequent_words),
            'rare_words': list(rare_words)[:10],
            'frequent_words': list(frequent_words)
        }
    
    def remove_noise_patterns(self, text: str) -> str:
        """Elimina patrones de ruido usando regex (menos agresivo)"""
        if not text:
            return text
        
        original_text = text
        
        # Solo aplicar patrones muy específicos
        for pattern in self.noise_patterns[:1]:  # Solo el primer patrón (palabras muy cortas)
            text = re.sub(pattern, ' ', text)
        
        # Normalizar espacios
        text = re.sub(r'\s+', ' ', text).strip()
        
        # Si el resultado es muy diferente, devolver original
        if len(text) < len(original_text) * 0.7:
            return original_text
        
        return text if text else original_text
    
    def remove_noise_words(self, text: str) -> str:
        """Elimina palabras de ruido específicas"""
        if not text:
            return text
        
        words = text.split()
        filtered_words = [
            word for word in words 
            if word.lower() not in self.noise_words
        ]
        return ' '.join(filtered_words) if filtered_words else text
    
    def remove_short_words(self, text: str, min_length: int = 2) -> str:
        """Elimina palabras muy cortas (menos agresivo)"""
        if not text:
            return text
        
        words = text.split()
        filtered_words = []
        
        for word in words:
            # Mantener palabras importantes aunque sean cortas
            if len(word) >= min_length or self._is_important_word(word):
                filtered_words.append(word)
        
        return ' '.join(filtered_words) if filtered_words else text
    
    def remove_non_alphabetic(self, text: str, keep_numbers: bool = True) -> str:
        """Elimina tokens que no son alfabéticos (más permisivo)"""
        if not text:
            return text
        
        words = text.split()
        filtered_words = []
        
        for word in words:
            if word.isalpha():
                filtered_words.append(word)
            elif keep_numbers and (word.isdigit() or self._contains_numbers(word)):
                filtered_words.append(word)
            elif self._is_important_word(word):
                filtered_words.append(word)
        
        return ' '.join(filtered_words) if filtered_words else text
    
    def _contains_numbers(self, word: str) -> bool:
        """Verifica si una palabra contiene números importantes"""
        return any(char.isdigit() for char in word) and len(word) <= 10
    
    def remove_by_pos(self, text: str, pos_to_remove: List[str] = None) -> str:
        """
        Elimina palabras según su categoría gramatical (menos agresivo)
        
        Args:
            text: Texto a procesar
            pos_to_remove: Lista de POS tags a eliminar
        """
        if not text:
            return text
        
        if pos_to_remove is None:
            # Solo eliminar categorías muy específicas y poco importantes
            pos_to_remove = ['DET', 'ADP']  # Solo determinantes y preposiciones
        
        try:
            doc = nlp(text)
            filtered_words = []
            
            for token in doc:
                if not token.is_space:
                    # Mantener palabras importantes sin importar su POS
                    if (self._is_important_word(token.text) or 
                        token.pos_ not in pos_to_remove):
                        filtered_words.append(token.text)
            
            result = ' '.join(filtered_words)
            return result if result.strip() else text
            
        except Exception as e:
            print(f"⚠️ Error en filtrado por POS: {e}")
            return text
    
    def remove_environmental_noise(self, text: str) -> str:
        """
        Elimina ruido específico del dominio ambiental (muy conservador)
        """
        if not text:
            return text
        
        # Solo palabras muy generales que realmente no aportan en contexto ambiental
        env_noise = {
            'general', 'específico', 'particular', 'especial', 'normal',
            'actual', 'presente', 'nuevo', 'viejo', 'grande', 'pequeño',
            'mejor', 'peor', 'principal', 'secundario'
        }
        
        words = text.split()
        filtered_words = []
        
        for word in words:
            # Solo eliminar si es ruido ambiental Y no es una palabra importante
            if (word.lower() not in env_noise or 
                self._is_important_word(word)):
                filtered_words.append(word)
        
        return ' '.join(filtered_words) if filtered_words else text
    
    def advanced_noise_removal(self, text: str) -> Dict:
        """
        Eliminación avanzada de ruido usando spaCy (menos agresiva)
        """
        if not text:
            return {
                'cleaned_text': text,
                'original_tokens': 0,
                'kept_tokens': 0,
                'removed_tokens': 0,
                'removal_details': []
            }
        
        try:
            doc = nlp(text)
            
            kept_tokens = []
            removed_tokens = []
            
            for token in doc:
                # Criterios más conservadores para eliminar
                should_remove = (
                    token.is_space or          # Es espacio
                    (token.is_punct and len(token.text) == 1) or  # Puntuación simple
                    (len(token.text) < 2 and not token.is_alpha) or  # Muy corto y no alfabético
                    (token.text.lower() in self.noise_words)  # Palabras de ruido específicas
                )
                
                # NUNCA eliminar palabras importantes
                if self._is_important_word(token.text):
                    should_remove = False
                
                if should_remove:
                    removed_tokens.append({
                        'text': token.text,
                        'reason': self._get_removal_reason(token)
                    })
                else:
                    # Usar el texto original en lugar del lema para preservar mejor el significado
                    kept_tokens.append(token.text)
            
            result_text = ' '.join(kept_tokens)
            
            # Verificar que el resultado tenga sentido
            if len(kept_tokens) < len([t for t in doc if not t.is_space]) * 0.5:
                print("⚠️ Eliminación avanzada demasiado agresiva, usando texto original")
                return {
                    'cleaned_text': text,
                    'original_tokens': len(doc),
                    'kept_tokens': len([t for t in doc if not t.is_space]),
                    'removed_tokens': 0,
                    'removal_details': []
                }
            
            return {
                'cleaned_text': result_text,
                'original_tokens': len(doc),
                'kept_tokens': len(kept_tokens),
                'removed_tokens': len(removed_tokens),
                'removal_details': removed_tokens[:20]
            }
            
        except Exception as e:
            print(f"⚠️ Error en eliminación avanzada: {e}")
            return {
                'cleaned_text': text,
                'original_tokens': 0,
                'kept_tokens': 0,
                'removed_tokens': 0,
                'removal_details': []
            }
    
    def _get_removal_reason(self, token) -> str:
        """Determina la razón por la cual se elimina un token"""
        if token.is_space:
            return 'whitespace'
        elif token.is_punct and len(token.text) == 1:
            return 'simple_punctuation'
        elif len(token.text) < 2 and not token.is_alpha:
            return 'too_short_non_alpha'
        elif token.text.lower() in self.noise_words:
            return 'noise_word'
        else:
            return 'other'
    
    def comprehensive_noise_removal(self, text: str, options: Dict = None) -> Dict:
        """
        Eliminación completa de ruido con múltiples estrategias (CORREGIDA)
        
        Args:
            text: Texto a limpiar
            options: Opciones de limpieza
        
        Returns:
            Diccionario con texto limpio y estadísticas
        """
        if options is None:
            # Configuración más conservadora por defecto
            options = {
                'remove_stopwords': True,
                'aggressive_stopwords': False,  # Nuevo parámetro
                'remove_short_words': True,
                'min_word_length': 2,  # Reducido de 3 a 2
                'remove_noise_words': True,
                'remove_by_frequency': False,  # Desactivado por defecto para textos cortos
                'min_frequency': 1,
                'max_frequency_ratio': 0.7,  # Más permisivo
                'remove_non_alphabetic': False,  # Desactivado por defecto
                'keep_numbers': True,
                'remove_by_pos': False,  # Desactivado por defecto
                'remove_environmental_noise': False,  # Desactivado por defecto
                'use_advanced_removal': False  # Desactivado por defecto
            }
        
        # Validar entrada
        if not text or not isinstance(text, str):
            return {
                'original_text': text or '',
                'cleaned_text': text or '',
                'original_word_count': 0,
                'final_word_count': 0,
                'words_removed': 0,
                'reduction_percentage': 0,
                'processing_steps': ['invalid_input'],
                'options_used': options
            }
        
        original_text = text.strip()
        if len(original_text) == 0:
            return {
                'original_text': original_text,
                'cleaned_text': original_text,
                'original_word_count': 0,
                'final_word_count': 0,
                'words_removed': 0,
                'reduction_percentage': 0,
                'processing_steps': ['empty_text'],
                'options_used': options
            }
        
        current_text = original_text
        processing_steps = []
        
        # Determinar agresividad basada en longitud del texto
        word_count = len(current_text.split())
        is_short_text = word_count <= 10
        
        try:
            # Paso 1: Eliminar palabras de ruido específicas (siempre seguro)
            if options.get('remove_noise_words', True):
                current_text = self.remove_noise_words(current_text)
                processing_steps.append('noise_words')
            
            # Paso 2: Eliminar palabras muy cortas (conservador)
            if options.get('remove_short_words', True):
                min_length = options.get('min_word_length', 2)
                current_text = self.remove_short_words(current_text, min_length)
                processing_steps.append('short_words')
            
            # Paso 3: Eliminar stopwords (adaptativo según longitud)
            if options.get('remove_stopwords', True) and not is_short_text:
                aggressive = options.get('aggressive_stopwords', False) and word_count > 20
                current_text = self.remove_stopwords(current_text, aggressive=aggressive)
                processing_steps.append('stopwords')
            
            # Paso 4: Filtrado por frecuencia (solo para textos largos)
            if (options.get('remove_by_frequency', False) and 
                word_count > 20):
                min_freq = options.get('min_frequency', 1)
                max_ratio = options.get('max_frequency_ratio', 0.7)
                freq_result = self.remove_by_frequency(current_text, min_freq, max_ratio)
                current_text = freq_result['filtered_text']
                processing_steps.append('frequency_filtering')
            
            # Paso 5: Eliminar no alfabéticos (opcional)
            if options.get('remove_non_alphabetic', False):
                keep_nums = options.get('keep_numbers', True)
                current_text = self.remove_non_alphabetic(current_text, keep_nums)
                processing_steps.append('non_alphabetic')
            
            # Paso 6: Filtrado por POS (opcional y conservador)
            if options.get('remove_by_pos', False) and word_count > 15:
                current_text = self.remove_by_pos(current_text)
                processing_steps.append('pos_filtering')
            
            # Paso 7: Eliminar ruido ambiental (opcional)
            if options.get('remove_environmental_noise', False):
                current_text = self.remove_environmental_noise(current_text)
                processing_steps.append('environmental_noise')
            
            # Paso 8: Eliminación avanzada (opcional)
            if options.get('use_advanced_removal', False):
                advanced_result = self.advanced_noise_removal(current_text)
                current_text = advanced_result['cleaned_text']
                processing_steps.append('advanced_removal')
            
            # Validación final: asegurar que el texto no esté vacío o sea incoherente
            if not current_text or len(current_text.strip()) == 0:
                print("⚠️ El procesamiento resultó en texto vacío, devolviendo texto original")
                current_text = original_text
                processing_steps = ['fallback_to_original']
            
            # Verificar que el texto resultante tenga sentido mínimo
            final_words = current_text.split()
            original_words = original_text.split()
            
            if len(final_words) < len(original_words) * 0.2:  # Si quedan menos del 20%
                print(f"⚠️ Eliminación demasiado agresiva ({len(final_words)}/{len(original_words)}), devolviendo texto original")
                current_text = original_text
                processing_steps = ['too_aggressive_fallback']
            
            # Calcular estadísticas finales
            original_word_count = len(original_words)
            final_word_count = len(final_words)
            words_removed = original_word_count - final_word_count
            reduction_percentage = (words_removed / original_word_count) * 100 if original_word_count > 0 else 0
            
            result = {
                'original_text': original_text,
                'cleaned_text': current_text,
                'original_word_count': original_word_count,
                'final_word_count': final_word_count,
                'words_removed': words_removed,
                'reduction_percentage': round(reduction_percentage, 2),
                'processing_steps': processing_steps,
                'options_used': options
            }
            
            return result
            
        except Exception as e:
            print(f"❌ Error durante eliminación de ruido: {e}")
            # Devolver texto original en caso de error
            return {
                'original_text': original_text,
                'cleaned_text': original_text,
                'original_word_count': len(original_text.split()),
                'final_word_count': len(original_text.split()),
                'words_removed': 0,
                'reduction_percentage': 0,
                'processing_steps': ['error_fallback'],
                'options_used': options,
                'error': str(e)
            }

# Ejemplo de uso del módulo corregido
if __name__ == "__main__":
    print("=== MÓDULO 5: ELIMINACIÓN DE RUIDO (CORREGIDO) ===")
    
    # Crear instancia del eliminador de ruido
    noise_remover = NoiseRemover()
    
    # Textos de prueba
    test_texts = [
        "Las plantas ayudan a limpiar el aire que respiramos todos los días",
        "El cambio climático es un problema muy serio que afecta a todo el mundo",
        "Bueno, pues, el tema del cambio climático es, eh, muy importante y fundamental para el futuro",
        "La energía solar es una alternativa limpia y renovable"
    ]
    
    for i, text in enumerate(test_texts, 1):
        print(f"\n--- PRUEBA {i} ---")
        print(f"Texto original: '{text}'")
        
        # Procesamiento conservador (por defecto)
        result = noise_remover.comprehensive_noise_removal(text)
        print(f"Texto limpio: '{result['cleaned_text']}'")
        print(f"Reducción: {result['reduction_percentage']:.1f}%")
        print(f"Pasos: {', '.join(result['processing_steps'])}")
        
        # Verificar que el resultado tenga sentido
        if result['cleaned_text'] and len(result['cleaned_text'].strip()) > 0:
            print("✓ Procesamiento exitoso")
        else:
            print("❌ Procesamiento falló - texto vacío")
    
    print(f"\n✨ MÓDULO DE ELIMINACIÓN DE RUIDO CORREGIDO ✨")


=== MÓDULO 5: ELIMINACIÓN DE RUIDO (CORREGIDO) ===

--- PRUEBA 1 ---
Texto original: 'Las plantas ayudan a limpiar el aire que respiramos todos los días'
Texto limpio: 'plantas ayudan limpiar aire respiramos todos días'
Reducción: 41.7%
Pasos: noise_words, short_words, stopwords
✓ Procesamiento exitoso

--- PRUEBA 2 ---
Texto original: 'El cambio climático es un problema muy serio que afecta a todo el mundo'
Texto limpio: 'cambio climático problema serio afecta todo mundo'
Reducción: 50.0%
Pasos: noise_words, short_words, stopwords
✓ Procesamiento exitoso

--- PRUEBA 3 ---
Texto original: 'Bueno, pues, el tema del cambio climático es, eh, muy importante y fundamental para el futuro'
Texto limpio: 'Bueno, pues, tema cambio climático es, eh, importante fundamental para futuro'
Reducción: 31.2%
Pasos: noise_words, short_words, stopwords
✓ Procesamiento exitoso

--- PRUEBA 4 ---
Texto original: 'La energía solar es una alternativa limpia y renovable'
Texto limpio: 'La energía solar es una 

Modulo 6: Lematizacion y Stemming

In [203]:
class BERTProcessor:
    def __init__(self, model_name: str = 'dccuchile/bert-base-spanish-wwm-uncased'):
        self.model_name = model_name
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.tokenizer = None
        self.model = None
        self.max_length = 512
        
        # Términos ambientales clave y sus contextos asociados
        self.environmental_terms = {
            'plantas': {
                'context': 'a través del proceso de fotosíntesis',
                'scientific': 'como biofiltros naturales',
                'impact': 'contribuyendo al equilibrio ecológico'
            },
            'aire': {
                'context': 'eliminando contaminantes y produciendo oxígeno',
                'scientific': 'mejorando la calidad atmosférica',
                'impact': 'reduciendo enfermedades respiratorias'
            },
            'respirar': {
                'context': 'mejorando así nuestra salud respiratoria',
                'scientific': 'optimizando el intercambio gaseoso',
                'impact': 'aumentando el bienestar general'
            }
        }
        
        # Configuración de mejora por tipo de texto
        self.improvement_templates = {
            'descripcion': {
                'intro': "¿Sabías que {text}",
                'outro': "Este proceso natural es esencial para nuestro ecosistema."
            },
            'contenido': {
                'intro': "Estudios demuestran que {text}",
                'outro': "Estos beneficios ecológicos son fundamentales para la sostenibilidad."
            }
        }

    def _load_model(self):
        """Carga el modelo BERT si no está cargado"""
        if self.tokenizer is None or self.model is None:
            try:
                print(f"⏳ Cargando modelo BERT: {self.model_name}")
                self.tokenizer = BertTokenizer.from_pretrained(self.model_name)
                self.model = BertModel.from_pretrained(self.model_name).to(self.device)
                print("✓ Modelo cargado exitosamente")
            except Exception as e:
                print(f"⚠️ Error cargando modelo: {str(e)}")
                print("🔁 Intentando con modelo alternativo...")
                self.model_name = 'bert-base-multilingual-cased'
                self.tokenizer = BertTokenizer.from_pretrained(self.model_name)
                self.model = BertModel.from_pretrained(self.model_name).to(self.device)

    def improve_text_with_context(self, text: str, improvement_type: str = 'descripcion', style: str = 'balanced') -> Dict:
        """
        Mejora el texto con contexto ambiental, adaptándose al tipo y estilo solicitado
        """
            # Validar el parámetro style
        valid_styles = ['balanced', 'technical', 'conservative']
        if style not in valid_styles:
            style = 'balanced'  # Valor por defecto si no es válido
            print(f"⚠️ Estilo '{style}' no válido. Usando 'balanced' por defecto")
            
        self._load_model()
        
        # Paso 1: Análisis inicial del texto
        original_embedding = self.generate_embeddings(text)
        original_terms = self._detect_environmental_terms(text)
        
        # Paso 2: Generar versiones mejoradas
        versions = []
        
        # Versión básica (conservadora)
        basic_improved = self._basic_improvement(text, improvement_type)
        versions.append({
            'type': 'basic',
            'text': basic_improved,
            'style': 'conservative'
        })
        
        # Versión con contexto ambiental
        if original_terms:
            env_improved = self._add_environmental_context(text, style)
            versions.append({
                'type': 'environmental',
                'text': env_improved,
                'style': 'environmental'
            })
        
        # Versión técnica (si hay términos detectados)
        if len(original_terms) >= 2:
            tech_improved = self._technical_improvement(text)
            versions.append({
                'type': 'technical',
                'text': tech_improved,
                'style': 'technical'
            })
        
        # Paso 3: Evaluar y seleccionar la mejor versión
        evaluated_versions = []
        for version in versions:
            similarity = self.calculate_semantic_similarity(text, version['text'])
            evaluated_versions.append({
                **version,
                'similarity': similarity['average_similarity'],
                'env_score': self._calculate_environmental_score(version['text']),
                'readability': self._calculate_readability(version['text'])
            })
        
        # Seleccionar la mejor versión según el estilo solicitado
        if style == 'balanced':
            best_version = max(evaluated_versions, key=lambda x: x['similarity'] + x['env_score'])
        elif style == 'technical':
            best_version = max(evaluated_versions, key=lambda x: x['env_score'])
        else:  # conservative
            best_version = max(evaluated_versions, key=lambda x: x['similarity'])
        
        return {
            'original_text': text,
            'improved_text': best_version['text'],
            'improvement_score': best_version['similarity'],
            'environmental_score': best_version['env_score'],
            'readability_score': best_version['readability'],
            'all_versions': evaluated_versions,
            'selected_style': best_version['style']
        }

    def _basic_improvement(self, text: str, text_type: str) -> str:
        """Mejora básica de estructura y fluidez"""
        template = self.improvement_templates.get(text_type, {})
        
        improved = text.capitalize()
        if 'intro' in template:
            improved = template['intro'].format(text=improved)
        if 'outro' in template and len(text.split()) < 25:
            improved += " " + template['outro']
        
        # Corrección gramatical básica
        improved = improved.replace(" todos los días", " diariamente")
        improved = improved.replace(" ayudan a ", " contribuyen a ")
        
        return improved

    def _add_environmental_context(self, text: str, style: str = 'balanced') -> str:
        """Añade contexto ambiental relevante según el estilo"""
        words = text.lower().split()
        improved = text
        
        # Validar el parámetro style
        valid_styles = ['technical', 'impact', 'balanced']
        if style not in valid_styles:
            style = 'balanced'  # Valor por defecto si no es válido
        
        for i, word in enumerate(words):
            if word in self.environmental_terms:
                context_info = self.environmental_terms[word]
                
                if style == 'technical':
                    addition = context_info.get('scientific', context_info['context'])
                elif style == 'impact':
                    addition = context_info.get('impact', context_info['context'])
                else:  # balanced
                    addition = context_info['context']
                
                # Insertar en la posición correcta manteniendo capitalización
                original_word = text.split()[i]
                improved = improved.replace(original_word, f"{original_word} {addition}", 1)
        
        return improved


    def _technical_improvement(self, text: str) -> str:
        """Crea una versión técnica del texto"""
        technical_terms = {
            'ayudan': 'contribuyen significativamente a',
            'limpian': 'depuran y purifican',
            'aire': 'la composición atmosférica',
            'días': 'el ciclo diario'
        }
        
        # Reemplazar términos básicos
        technical_text = text
        for term, replacement in technical_terms.items():
            technical_text = technical_text.replace(term, replacement)
        
        # Añadir datos científicos si es posible
        if 'plantas' in technical_text.lower():
            technical_text += " mediante procesos bioquímicos esenciales"
        
        return technical_text

    def _detect_environmental_terms(self, text: str) -> List[str]:
        """Detecta términos ambientales en el texto"""
        return [term for term in self.environmental_terms if term in text.lower()]

    def _calculate_readability(self, text: str) -> float:
        """Calcula un score de legibilidad (simplificado)"""
        words = text.split()
        avg_word_len = sum(len(word) for word in words) / len(words) if words else 0
        sentence_count = text.count('.') + text.count('!') + text.count('?')
        
        # Fórmula simplificada (mayor score = más legible)
        score = 10 - (avg_word_len / 10) - (len(words) / (sentence_count * 25) if sentence_count > 0 else 0)
        return max(1, min(10, score))

    def comprehensive_text_improvement(self, text: str, target_type: str = 'descripcion', options: Dict = None) -> Dict:
        """
        Interfaz principal para la mejora de texto con todas las capacidades
        """
        if options is None:
            options = {
                'style': 'balanced',  # balanced, technical, conservative
                'environmental_focus': True,
                'length_optimization': True
            }
        
        # Paso 1: Mejora básica
        result = self.improve_text_with_context(
            text,
            improvement_type=target_type,
            style=options['style']
        )
        
        # Paso 2: Optimización de longitud si es necesario
        if options['length_optimization']:
            optimized = self._optimize_length(result['improved_text'], target_type)
            if optimized != result['improved_text']:
                similarity = self.calculate_semantic_similarity(result['improved_text'], optimized)
                result['improved_text'] = optimized
                result['improvement_score'] = (result['improvement_score'] + similarity['average_similarity']) / 2
        
        # Paso 3: Generar reporte completo
        original_length = len(text.split())
        improved_length = len(result['improved_text'].split())
        
        return {
            'original_text': text,
            'improved_text': result['improved_text'],
            'improvement_details': {
                'quality_metrics': {
                    'semantic_similarity': round(result['improvement_score'] * 10, 1),
                    'environmental_relevance': round(result['environmental_score'], 1),
                    'readability': round(result['readability_score'], 1)
                },
                'length_analysis': {
                    'original': original_length,
                    'improved': improved_length,
                    'change': improved_length - original_length
                },
                'style_used': result['selected_style'],
                'environmental_terms_added': result['environmental_score'] - self._calculate_environmental_score(text)
            },
            'all_versions': result.get('all_versions', [])
        }

    def _optimize_length(self, text: str, text_type: str) -> str:
        """Optimiza la longitud del texto según su tipo"""
        words = text.split()
        word_count = len(words)
        
        # Objetivos de longitud por tipo de texto
        targets = {
            'descripcion': (15, 30),
            'contenido': (30, 50),
            'titulo': (5, 10)
        }.get(text_type, (20, 40))
        
        min_target, max_target = targets
        
        if word_count < min_target:
            # Texto demasiado corto - expandir
            expansions = {
                'descripcion': " Este proceso es fundamental para el equilibrio ecológico.",
                'contenido': " Investigaciones científicas respaldan estos beneficios ambientales.",
                'default': " Un aspecto clave para la sostenibilidad ambiental."
            }
            expansion = expansions.get(text_type, expansions['default'])
            return text + expansion
        
        elif word_count > max_target:
            # Texto demasiado largo - resumir
            if text_type == 'descripcion':
                return ' '.join(words[:max_target]) + '...'
            else:
                sentences = text.split('.')
                if len(sentences) > 1:
                    return sentences[0] + '.'
                return ' '.join(words[:max_target])
        
        return text

    def generate_embeddings(self, text: str) -> Dict:
        """Genera embeddings de texto usando BERT"""
        self._load_model()
        
        inputs = self.tokenizer(
            text, 
            return_tensors='pt',
            max_length=self.max_length,
            truncation=True,
            padding=True
        ).to(self.device)
        
        with torch.no_grad():
            outputs = self.model(**inputs)
        
        # Usar el embedding promedio de todos los tokens
        embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
        
        return {
            'embeddings': embeddings,
            'text': text,
            'token_count': inputs['input_ids'].shape[1]
        }

    def calculate_semantic_similarity(self, text1: str, text2: str) -> Dict:
        """Calcula la similitud semántica entre dos textos"""
        emb1 = self.generate_embeddings(text1)['embeddings']
        emb2 = self.generate_embeddings(text2)['embeddings']
        
        similarity = np.dot(emb1, emb2.T) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
        
        return {
            'average_similarity': float(similarity[0][0]),
            'text1': text1,
            'text2': text2
        }

    def _calculate_environmental_score(self, text: str) -> float:
        """Calcula un score de relevancia ambiental (0-10)"""
        terms_found = self._detect_environmental_terms(text)
        words = text.split()
        
        if not words:
            return 0.0
            
        # Score base por términos encontrados
        base_score = len(terms_found) * 2.0
        
        # Bonus por densidad de términos ambientales
        density = len(terms_found) / len(words)
        density_bonus = min(3.0, density * 10)
        
        return min(10.0, base_score + density_bonus)

Modulo 7: Procesamiento con BERT

In [204]:
def comprehensive_text_improvement(self, text: str, target_type: str = 'contenido', options: Dict = None) -> Dict:
    """
    Mejora completa del texto usando todas las capacidades de BERT
    """
    if options is None:
        options = {
            'environmental_focus': True,
            'generate_variations': True,
            'optimize_length': True,
            'include_embeddings': False,
            'similarity_threshold': 0.7
        }
    
    # Análisis inicial
    original_embeddings = self.generate_embeddings(text) if options.get('include_embeddings') else None
    
    # Mejora principal
    main_improvement = self.improve_text_with_context_enhanced(
        text, 
        improvement_type=target_type, 
        environmental_focus=options.get('environmental_focus', True)
    )
    
    # Resultado final
    final_text = main_improvement['best_improvement']['text']
    
    return {
        'original_text': text,
        'target_type': target_type,
        'options': options,
        'main_improvement': main_improvement,
        'variations': None,
        'final_improved_text': final_text,
        'improvement_summary': {
            'original_length': len(text.split()),
            'final_length': len(final_text.split()),
            'environmental_terms_added': self._calculate_environmental_score(final_text) - self._calculate_environmental_score(text),
            'semantic_preservation': main_improvement['improvement_score']
        },
        'original_embeddings': original_embeddings
    }

def _calculate_environmental_score(self, text: str) -> int:
    """
    Calcula la puntuación ambiental basada en la presencia de términos ambientales
    """
    text_lower = text.lower()
    return sum(1 for term in self.environmental_keywords if term.lower() in text_lower)

def generate_embeddings(self, text: str) -> Dict:
    """
    Genera embeddings para el texto usando BERT
    """
    # Inicializar modelo si no está cargado
    if self.tokenizer is None or self.model is None:
        try:
            self.tokenizer = BertTokenizer.from_pretrained(self.model_name)
            self.model = BertModel.from_pretrained(self.model_name).to(self.device)
        except Exception as e:
            # Fallback a un modelo más pequeño si hay problemas
            print(f"⚠️ Error cargando modelo {self.model_name}: {str(e)}")
            self.model_name = 'dccuchile/bert-base-spanish-wwm-uncased'
            self.tokenizer = BertTokenizer.from_pretrained(self.model_name)
            self.model = BertModel.from_pretrained(self.model_name).to(self.device)
    
    # Tokenizar y obtener embeddings
    tokens = self.tokenizer(text, return_tensors='pt', truncation=True, 
                           max_length=self.max_length, padding=True)
    
    # Mover tokens a GPU si está disponible
    input_ids = tokens['input_ids'].to(self.device)
    attention_mask = tokens['attention_mask'].to(self.device)
    
    # Obtener embeddings
    with torch.no_grad():
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
    
    # Usar el embedding del token [CLS] como representación del texto
    embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
    
    return {
        'embeddings': embeddings,
        'token_count': len(tokens['input_ids'][0]),
        'text': text
    }

def calculate_semantic_similarity(self, text1: str, text2: str) -> Dict:
    """
    Calcula la similitud semántica entre dos textos usando embeddings BERT
    """
    # Obtener embeddings
    embeddings1 = self.generate_embeddings(text1)['embeddings']
    embeddings2 = self.generate_embeddings(text2)['embeddings']
    
    # Calcular similitud coseno
    similarity = np.dot(embeddings1, embeddings2.T) / (
        np.linalg.norm(embeddings1) * np.linalg.norm(embeddings2)
    )
    
    return {
        'average_similarity': float(similarity[0][0]),
        'text1': text1,
        'text2': text2
    }


Modulo 8: Sistema Completo

In [205]:
# ===== SISTEMA COMPLETO MEJORADO (COMPLETO) =====

class TextMiningSystem:
    """
    Sistema completo de minería de texto que integra todos los módulos
    """
    
    def __init__(self):
        self.ingestion = TextIngestion()
        self.cleaner = TextCleaner()
        self.tokenizer = TextTokenizer()
        self.normalizer = TextNormalizer()
        self.noise_remover = NoiseRemover()
        self.lemmatizer = TextLemmatizer()
        self.bert_processor = BERTProcessor()
    
    def _validate_text(self, text, step_name=""):
        """Valida que el texto no sea None o vacío"""
        if text is None:
            print(f"⚠️ Texto es None en el paso: {step_name}")
            return None
        if not isinstance(text, str):
            print(f"⚠️ Texto debe ser string en el paso: {step_name}, recibido: {type(text)}")
            return None
        if len(text.strip()) == 0:
            print(f"⚠️ Advertencia: Texto vacío en el paso: {step_name}")
            return None
        return text.strip()
    
    def _create_fallback_result(self, text: str, content_type: str, error_msg: str) -> Dict:
        """Crea un resultado de fallback en caso de error"""
        return {
            'original_text': text,
            'final_text': text,
            'content_type': content_type,
            'intermediate_results': {},
            'processing_steps': [],
            'evaluation': {
                'semantic_similarity': 10.0,
                'coherence': 7.0,
                'grammar': 7.0,
                'environmental_relevance': 5.0,
                'readability': 7.0,
                'length_optimization': 7.0,
                'overall_quality': 7.0
            },
            'recommendations': [f"Error durante el procesamiento: {error_msg}"],
            'bert_details': None,
            'error': error_msg
        }
    
    def process_text_complete_enhanced(self, text: str, content_type: str = 'contenido', track_steps: bool = False) -> Dict:
        """Procesamiento mejorado con mejor control de calidad"""
        
        # Configuraciones específicas por tipo de contenido
        processing_configs = {
            'titulo': {
                'aggressive_cleaning': False,
                'preserve_length': True,
                'min_similarity': 0.7
            },
            'descripcion': {
                'aggressive_cleaning': False,
                'preserve_length': False,
                'min_similarity': 0.6
            },
            'contenido': {
                'aggressive_cleaning': True,
                'preserve_length': False,
                'min_similarity': 0.5
            }
        }
        
        config = processing_configs.get(content_type, processing_configs['contenido'])
        
        # Validación inicial
        original_input = text
        validated_text = self._validate_text(text, "entrada inicial")
        if validated_text is None:
            text = original_input
        else:
            text = validated_text
        
        intermediate_results = {}
        processing_steps = []
        
        try:
            # Paso 1: Ingesta
            print("Ejecutando Paso 1: Ingesta...")
            ingestion_result = self.ingestion.ingest_manual_text(text)
            current_text = ingestion_result['original_text']
            
            if track_steps:
                intermediate_results['ingestion'] = current_text
                processing_steps.append({
                    'step': 'ingesta',
                    'metrics': {'length': ingestion_result['length']}
                })
            
            # Paso 2: Limpieza (menos agresiva)
            print("Ejecutando Paso 2: Limpieza...")
            cleaning_options = {
                'remove_urls': True,
                'remove_emails': True,
                'remove_phones': True,
                'remove_html': True,
                'remove_special_chars': not config['preserve_length'],
                'keep_punctuation': True,
                'normalize_whitespace': True,
                'normalize_punctuation': True,
                'remove_newlines': True,
                'fix_encoding': True
            }
            cleaning_result = self.cleaner.basic_clean(current_text, cleaning_options)
            current_text = cleaning_result['cleaned_text']
            
            if track_steps:
                intermediate_results['cleaning'] = current_text
                processing_steps.append({
                    'step': 'limpieza',
                    'metrics': {'reduction_percentage': cleaning_result['reduction_percentage']}
                })
            
            # Paso 3: Tokenización
            print("Ejecutando Paso 3: Tokenización...")
            tokens = self.tokenizer.tokenize_words(current_text, 'spacy')
            if not tokens:
                tokens = current_text.split()
            
            if track_steps:
                intermediate_results['tokenization'] = ' '.join(tokens[:20]) + ('...' if len(tokens) > 20 else '')
                processing_steps.append({
                    'step': 'tokenización',
                    'metrics': {'token_count': len(tokens)}
                })
            
            # Paso 4: Normalización (mejorada)
            print("Ejecutando Paso 4: Normalización...")
            normalization_options = {
                'to_lowercase': True,
                'remove_accents': False,  # Preservar acentos para mejor legibilidad
                'expand_contractions': True,
                'expand_abbreviations': True,
                'normalize_numbers': 'keep',
                'normalize_case_patterns': True,
                'normalize_environmental_terms': True
            }
            normalization_result = self.normalizer.normalize_tokens(tokens, normalization_options)
            normalized_tokens = normalization_result.get('normalized_tokens', tokens)
            current_text = ' '.join(normalized_tokens)
            
            if track_steps:
                intermediate_results['normalization'] = current_text
                processing_steps.append({
                    'step': 'normalización',
                    'metrics': {'token_count_change': normalization_result.get('token_count_change', 0)}
                })
            
            # Paso 5: Eliminación de ruido (configuración adaptativa)
            print("Ejecutando Paso 5: Eliminación de ruido...")
            noise_options = {
                'remove_stopwords': config['aggressive_cleaning'],
                'aggressive_stopwords': False,
                'remove_short_words': True,
                'min_word_length': 2,
                'remove_noise_words': True,
                'remove_by_frequency': False,
                'remove_non_alphabetic': False,
                'remove_by_pos': False,
                'remove_environmental_noise': False,
                'use_advanced_removal': False
            }
            noise_removal_result = self.noise_remover.comprehensive_noise_removal(current_text, noise_options)
            current_text = noise_removal_result['cleaned_text']
            
            # Validar que el texto no esté vacío después de la eliminación de ruido
            if not current_text or len(current_text.strip()) == 0:
                print("⚠️ Texto vacío después de eliminación de ruido, usando texto anterior")
                current_text = ' '.join(normalized_tokens)
            
            if track_steps:
                intermediate_results['noise_removal'] = current_text
                processing_steps.append({
                    'step': 'eliminación_ruido',
                    'metrics': {'reduction_percentage': noise_removal_result['reduction_percentage']}
                })
            
            # Paso 6: Lematización (usando método existente)
            print("Ejecutando Paso 6: Lematización...")
            lemmatization_options = {
                'preserve_environmental_terms': True,
                'min_word_length': 2,
                'remove_duplicates': False
            }
            lemmatization_result = self.lemmatizer.comprehensive_processing(
                current_text, 
                method='spacy',
                options=lemmatization_options
            )
            current_text = lemmatization_result['final_processed_text']
            
            # Validar resultado de lematización
            if not current_text or len(current_text.strip()) == 0:
                print("⚠️ Lematización resultó en texto vacío, usando texto anterior")
                current_text = noise_removal_result['cleaned_text']
            
            if track_steps:
                intermediate_results['lemmatization'] = current_text
                processing_steps.append({
                    'step': 'lematización',
                    'metrics': {'changes_made': lemmatization_result.get('changes_made', 0)}
                })
            
            # Paso 7: Procesamiento con BERT (mejorado)
            print("Ejecutando Paso 7: Procesamiento con BERT...")
            
            # Validar texto antes de BERT
            if not current_text or len(current_text.strip()) == 0:
                print("⚠️ Texto vacío antes de BERT, usando texto original")
                current_text = text
            
            bert_options = {
                'environmental_focus': True,
                'generate_variations': False,
                'optimize_length': True,
                'preserve_meaning': True,
                'min_similarity_threshold': config['min_similarity']
            }
            
            # Usar método existente de BERT
            bert_result = self.bert_processor.comprehensive_text_improvement(
                current_text,
                target_type=content_type,
                options=bert_options
            )
            final_text = bert_result['final_improved_text']
            
            if track_steps:
                intermediate_results['bert_processing'] = final_text
                processing_steps.append({
                    'step': 'bert_processing',
                    'metrics': {
                        'semantic_preservation': bert_result['improvement_summary'].get('semantic_preservation', 0.7)
                    }
                })
            
            # Evaluación de calidad mejorada
            print("Ejecutando evaluación de calidad...")
            evaluation = self._evaluate_quality_enhanced(text, final_text, bert_result['improvement_summary'])
            
            # Generar recomendaciones
            recommendations = self._generate_recommendations_enhanced(evaluation, processing_steps)
            
            return {
                'original_text': text,
                'final_text': final_text,
                'content_type': content_type,
                'intermediate_results': intermediate_results if track_steps else {},
                'processing_steps': processing_steps if track_steps else [],
                'evaluation': evaluation,
                'recommendations': recommendations,
                'bert_details': bert_result,
                'processing_config': config
            }
            
        except Exception as e:
            print(f"❌ Error durante el procesamiento: {str(e)}")
            return self._create_fallback_result(text, content_type, str(e))
    
    def _evaluate_quality_enhanced(self, original_text: str, final_text: str, improvement_summary: Dict) -> Dict:
        """Evaluación de calidad mejorada"""
        try:
            original_words = len(original_text.split()) if original_text else 0
            final_words = len(final_text.split()) if final_text else 0
            
            # Calcular similitud semántica usando BERT
            try:
                similarity = self.bert_processor.calculate_semantic_similarity(original_text, final_text)
                semantic_similarity = similarity['average_similarity']
            except Exception as e:
                print(f"⚠️ Error calculando similitud: {e}")
                semantic_similarity = 0.7
            
            # Evaluar coherencia básica
            coherence_score = self._calculate_coherence(final_text)
            
            # Evaluar gramática básica
            grammar_score = self._calculate_grammar(final_text)
            
            # Score ambiental
            try:
                env_score = self.bert_processor._calculate_environmental_score(final_text)
            except Exception as e:
                print(f"⚠️ Error calculando score ambiental: {e}")
                env_score = 5.0
            
            # Evaluación de legibilidad
            readability_score = self._calculate_readability(final_text)
            
            # Score general ponderado
            overall_quality = (
                semantic_similarity * 3 +
                coherence_score * 2.5 +
                grammar_score * 2 +
                (env_score / 10) * 1.5 +
                (readability_score / 10) * 1
            )
            
            return {
                'semantic_similarity': semantic_similarity * 10,
                'coherence': coherence_score * 10,
                'grammar': grammar_score * 10,
                'environmental_relevance': env_score,
                'readability': readability_score,
                'length_optimization': min(10, max(1, 10 - abs(final_words - 15) / 5)) if final_words > 0 else 5.0,
                'overall_quality': min(10, overall_quality)
            }
            
        except Exception as e:
            print(f"❌ Error en evaluación: {e}")
            return {
                'semantic_similarity': 7.0,
                'coherence': 7.0,
                'grammar': 7.0,
                'environmental_relevance': 5.0,
                'readability': 7.0,
                'length_optimization': 7.0,
                'overall_quality': 6.8
            }
    
    def _calculate_coherence(self, text: str) -> float:
        """Calcula un score de coherencia básico"""
        if not text or len(text.strip()) == 0:
            return 0.3
        
        words = text.split()
        
        # Verificaciones básicas de coherencia
        coherence_score = 0.5
        
        # Verificar longitud apropiada
        if 3 <= len(words) <= 25:
            coherence_score += 0.2
        
        # Verificar que tenga estructura básica
        try:
            doc = nlp(text)
            has_noun = any(token.pos_ == 'NOUN' for token in doc)
            has_verb = any(token.pos_ == 'VERB' for token in doc)
            
            if has_noun:
                coherence_score += 0.15
            if has_verb:
                coherence_score += 0.15
        except:
            pass
        
        return min(1.0, coherence_score)
    
    def _calculate_grammar(self, text: str) -> float:
        """Calcula un score de gramática básico"""
        if not text:
            return 0.3
        
        grammar_score = 0.6  # Base score
        
        # Verificar que no termine abruptamente
        if text.strip().endswith(('.', '!', '?')):
            grammar_score += 0.1
        
        # Verificar longitud apropiada
        words = text.split()
        if 3 <= len(words) <= 20:
            grammar_score += 0.1
        
        # Verificar que no tenga palabras sueltas sin contexto
        if len(words) >= 3:
            grammar_score += 0.1
        
        # Verificar que no tenga repeticiones excesivas
        unique_words = len(set(words))
        if unique_words / len(words) > 0.7:  # Al menos 70% de palabras únicas
            grammar_score += 0.1
        
        return min(1.0, grammar_score)
    
    def _calculate_readability(self, text: str) -> float:
        """Calcula un score de legibilidad básico"""
        if not text:
            return 5.0
        
        words = text.split()
        sentences = text.split('.')
        
        # Métricas básicas
        avg_words_per_sentence = len(words) / len(sentences) if sentences else len(words)
        avg_word_length = sum(len(word) for word in words) / len(words) if words else 0
        
        # Score basado en complejidad
        readability = 8.0  # Base score
        
        # Penalizar oraciones muy largas
        if avg_words_per_sentence > 20:
            readability -= 2
        elif avg_words_per_sentence > 15:
            readability -= 1
        
        # Penalizar palabras muy largas
        if avg_word_length > 8:
            readability -= 1.5
        elif avg_word_length > 6:
            readability -= 0.5
        
        # Bonificar longitud apropiada
        if 5 <= len(words) <= 20:
            readability += 1
        
        return max(1.0, min(10.0, readability))
    
    def _generate_recommendations_enhanced(self, evaluation: Dict, processing_steps: List) -> List[str]:
        """Genera recomendaciones mejoradas"""
        recommendations = []
        
        try:
            if evaluation.get('semantic_similarity', 0) < 7:
                recommendations.append("Ajustar parámetros de lematización para preservar mejor el significado original")
            
            if evaluation.get('coherence', 0) < 7:
                recommendations.append("Mejorar la coherencia textual manteniendo la estructura gramatical")
            
            if evaluation.get('grammar', 0) < 7:
                recommendations.append("Revisar la gramática del texto procesado")
            
            if evaluation.get('environmental_relevance', 0) < 5:
                recommendations.append("Incorporar más términos ambientales relevantes")
            
            if evaluation.get('readability', 0) < 6:
                recommendations.append("Simplificar el vocabulario para mejorar la legibilidad")
            
            if evaluation.get('length_optimization', 0) < 7:
                recommendations.append("Ajustar la longitud según el tipo de contenido")
            
            if not recommendations:
                recommendations.append("El texto ha sido procesado exitosamente con alta calidad")
                
        except Exception as e:
            recommendations.append(f"Error generando recomendaciones: {str(e)}")
        
        return recommendations
    
    # Mantener métodos originales para compatibilidad
    def process_text_complete(self, text: str, content_type: str = 'contenido', track_steps: bool = False) -> Dict:
        """Método original para compatibilidad"""
        return self.process_text_complete_enhanced(text, content_type, track_steps)
    
    def batch_process(self, texts: List[Dict], progress_callback=None) -> List[Dict]:
        """Procesa múltiples textos en lote"""
        results = []
        total = len(texts)
        
        for i, text_info in enumerate(texts):
            try:
                if progress_callback:
                    progress_callback(i + 1, total)
                
                if not isinstance(text_info, dict) or 'text' not in text_info:
                    continue
                
                text = text_info['text']
                if not text or not isinstance(text, str):
                    continue
                
                result = self.process_text_complete_enhanced(
                    text, 
                    text_info.get('content_type', 'contenido')
                )
                
                results.append({
                    'id': text_info.get('id', f'text_{i}'),
                    'content_type': text_info.get('content_type', 'contenido'),
                    'original': text,
                    'improved': result['final_text'],
                    'evaluation': result['evaluation'],
                    'success': True
                })
                
            except Exception as e:
                print(f"❌ Error procesando texto {i}: {e}")
                results.append({
                    'id': text_info.get('id', f'text_{i}'),
                    'content_type': text_info.get('content_type', 'contenido'),
                    'original': text_info.get('text', 'Error'),
                    'improved': text_info.get('text', 'Error'),
                    'evaluation': {'overall_quality': 0.0},
                    'success': False,
                    'error': str(e)
                })
        
        return results
    
    def generate_report(self, result: Dict) -> str:
        """Genera un reporte detallado del procesamiento"""
        try:
            report = f"# Reporte de Procesamiento de Texto\n\n"
            report += f"## Texto Original\n{result.get('original_text', 'No disponible')}\n\n"
            report += f"## Texto Mejorado\n{result.get('final_text', 'No disponible')}\n\n"
            report += f"## Tipo de Contenido\n{result.get('content_type', 'No especificado')}\n\n"
            
            report += f"## Evaluación de Calidad\n"
            evaluation = result.get('evaluation', {})
            for metric, score in evaluation.items():
                metric_name = metric.replace('_', ' ').title()
                report += f"- **{metric_name}**: {score:.2f}/10\n"
            
            report += f"\n## Recomendaciones\n"
            recommendations = result.get('recommendations', [])
            for rec in recommendations:
                report += f"- {rec}\n"
            
            return report
            
        except Exception as e:
            return f"Error generando reporte: {str(e)}"

# Ejemplo de uso del sistema mejorado
print("\n=== SISTEMA COMPLETO MEJORADO ===")

# Crear instancia del sistema mejorado
text_mining_system = TextMiningSystem()

# Probar con el texto problemático
test_text = "Las plantas ayudan a limpiar el aire que respiramos todos los días"

print(f"Procesando: '{test_text}'")
print("=" * 60)

try:
    result = text_mining_system.process_text_complete_enhanced(
        test_text,
        'descripcion',
        track_steps=True
    )

    print("RESULTADO FINAL:")
    print(f"Original: '{result['original_text']}'")
    print(f"Mejorado: '{result['final_text']}'")

    print(f"\nPasos intermedios:")
    for step_name, text in result['intermediate_results'].items():
        print(f"- {step_name}: '{text[:100]}{'...' if len(text) > 100 else ''}'")

    print(f"\nEVALUACIÓN:")
    for metric, score in result['evaluation'].items():
        print(f"- {metric.replace('_', ' ').title()}: {score:.2f}/10")

    print(f"\nRECOMENDACIONES:")
    for rec in result['recommendations']:
        print(f"- {rec}")

except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

print(f"\n✨ SISTEMA MEJORADO COMPLETADO ✨")



=== SISTEMA COMPLETO MEJORADO ===
Procesando: 'Las plantas ayudan a limpiar el aire que respiramos todos los días'
Ejecutando Paso 1: Ingesta...
Ejecutando Paso 2: Limpieza...
Ejecutando Paso 3: Tokenización...
Ejecutando Paso 4: Normalización...
Ejecutando Paso 5: Eliminación de ruido...
Ejecutando Paso 6: Lematización...
Ejecutando Paso 7: Procesamiento con BERT...
❌ Error durante el procesamiento: 'style'
RESULTADO FINAL:
Original: 'Las plantas ayudan a limpiar el aire que respiramos todos los días'
Mejorado: 'Las plantas ayudan a limpiar el aire que respiramos todos los días'

Pasos intermedios:

EVALUACIÓN:
- Semantic Similarity: 10.00/10
- Coherence: 7.00/10
- Grammar: 7.00/10
- Environmental Relevance: 5.00/10
- Readability: 7.00/10
- Length Optimization: 7.00/10
- Overall Quality: 7.00/10

RECOMENDACIONES:
- Error durante el procesamiento: 'style'

✨ SISTEMA MEJORADO COMPLETADO ✨
